# Using Bayesian Genetic Algorithms to autonomously solve mazes
### Jacob Sanz-Robinson

## Introduction and Plan

As their name suggests, genetic algorithms borrow characteristics from the concepts of biological evolution and natural selection to solve problems. They are based on the fundamental idea of having a population composed of randomly generated programs attempting to solve the given problem, and each program receiving a score for how it performed. The programs with the highest scores will have their “genetic information” combined, resulting in a new generation of programs, better suited to solving the challenge at hand than their parents. The “genetics” of the best performers of the new generation are combined, and so forth iteratively, with mutations sporadically occurring, until a solution is reached. This way the algorithm essentially learns to search for the solution to a problem all by itself.

The idea behind this project is to further improve upon the traditional framework of a Genetic Algorithm by introducing Bayesian Statistics to optimize the evolutive process. We take advantage of the information gained in the previous generation to generate better offspring by making use of Bayes theorem to estimate the posterior distribution of programs from their prior distribution and likelihood function. These are calculated based on the fitness for each individual in the population. We also accelerate evolution by actively selecting the best individuals to pass on their genes.

This project is partially based on a number of ideas presented in the paper:

*Zhang, Byong-Tak. (2000). Bayesian Methods for Efficient Genetic Programming. Genetic Programming and Evolvable Machines. 1. 217-242. 10.1023/A:1010010230007*.

It should be noted, however, that the selection mechanism proposed in the paper of replacing a non-selected individual with the array-position individual from the past generation didn't allow this particular Genetic Algorithm to converge. Instead I opted for replacing this selection mechanism with max-pooling, which consists of replacing non-selected individuals with individuals from the top 3% of the population. In this way the use of bayesian statistics consists of comparing the posterior distribution of the current and previous generation to know when to employ max pooling.

# Imports and parameters

In [14]:
import random
from operator import itemgetter
import math
import statistics
import time

In [2]:
#Denote start of a maze with a 9, and end with a 10.
#Wall = 1, path = 0.

maze = [[1,1,1,1,1,1,1,1,1], #9x9 sample maze.
        [1,9,0,1,1,1,0,1,1],
        [1,0,0,1,0,0,0,1,1],
        [1,1,0,0,0,1,1,1,1],
        [1,1,1,1,0,0,1,1,1],
        [1,0,0,0,0,1,1,1,1],
        [1,1,0,1,0,1,10,0,1],
        [1,0,0,1,0,0,0,0,1],
        [1,1,1,1,1,1,1,1,1]]

#big_maze = [[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1], #be warned, this takes very long to solve.
#            [1,1,1,0,0,0,0,0,0,0,1,1,1,1,1],
#            [1,1,1,0,1,1,0,0,1,0,0,0,1,1,1],
#            [1,1,1,0,0,0,0,1,1,0,1,0,1,1,1],
#            [1,1,1,1,1,1,0,0,0,0,1,0,1,1,1],
#            [1,1,1,1,1,0,0,1,1,0,0,0,0,1,1],
#            [1,0,0,1,1,0,1,1,1,1,1,1,0,1,1],
#            [1,0,9,0,1,0,1,0,0,1,1,1,0,1,1],
#            [1,0,1,1,1,0,1,0,0,1,1,0,0,1,1],
#            [1,0,1,1,1,0,0,0,1,1,1,0,1,1,1],
#            [1,0,0,0,1,1,1,0,1,1,1,0,1,1,1],
#            [1,1,1,0,1,1,1,0,1,1,10,0,1,1,1],
#            [1,1,1,0,0,0,0,0,1,1,1,1,1,1,1],
#            [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
#            [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]]

popSize = 500 #Population size 500
mutateRate = 20 #Mutation rate out of 100. The probability of a random mutation ocurring during breeding.
individualLen = 20 #How many genes in an individual.

# Common functions

These are basic functions to act on the population and maze of the genetic algorithm. The crossover function is used in the 'vanilla' version of the genetic algorithm, but not in the Bayesian version (they are redefined further on in the notebook).

The Manhattan distance between the last "living" location of a program and the end-point of the maze is used as a fitness function. This is the distance between two points measured along axes at right angles. It will also come in handy during the Bayesian implementation as it can act as a loss function for the generation of our likelihoods.

In [3]:
def generateIndividual(individualLen):
#Returns a list filled with random numbers 0 = up,1 = right,2 = down,3 = left.
    return [random.randrange(0,4) for i in range(individualLen)]

def finalCoor(maze, coorlist):
#Given a list of coordinates it visits in a maze, check if an individual succeeds or fails. 
#Returns the position where the invidual ended it's run at.
    for coor in coorlist:
        tocheckX = coor[0]
        tocheckY = coor[1]
        if maze[tocheckY][tocheckX] == 1:
            return coor #crashed into a wall
        if maze[tocheckY][tocheckX] == 10:
            return coor #successful
    return coor #didnt get there

def distance(maze, coorList):
#Manhattan Distance as fitness.
#It is important this is an error function when implementing the Bayesian version.
    newDistance = [None, None]
    newDistance[0] = abs(endPosition(maze)[0] - finalCoor(maze, coorList)[0])
    newDistance[1] = abs(endPosition(maze)[1] - finalCoor(maze, coorList)[1])
    return sum(newDistance)

def startingPosition(maze):
#Finds the coordinate of the starting position in a Maze.
#Coordinate x is from left to right, and coordinate y is from up-->down (so the upper row is y=0)
    count_y = -1
    coor = [0, 0]
    for row in maze:
        count_y += 1
        count_x = -1
        for col in row:
            count_x += 1
            if col == 9:
                coor[0] = count_x
                coor[1] = count_y
    return coor

def endPosition(maze):
#Finds the coordinate of the starting position in a Maze.
#Coordinate x is from left to right, and coordinate y is from up-->down (so the upper row is y=0)
    count_y = -1
    coor = [0, 0]
    for row in maze:
        count_y += 1
        count_x = -1
        for col in row:
            count_x += 1
            if col == 10:
                coor[0] = count_x
                coor[1] = count_y
    return coor

def topFracPop(orderedPopulationList, denominator): 
#Given a population list [[[individual], fitness],...] ordered by fitness
#Returns the top fraction of the population's individuals. Eg denominator = 2.0 to get top half.
    topFrac = []
    count = 0
    while count < int((len(orderedPopulationList)/denominator)): 
        topFrac.append(orderedPopulationList[count][0])
        count += 1
    return topFrac

def allCoordsVisited(maze, individual):
#Generates all the coordinates an individual would visit (in order) if it didn't run into a wall or out of bounds.
    length = len(individual) + 1
    coordList = [[0] * 2 for i in range(length)]
    coordList[0] = startingPosition(maze)
    chromosome = 0
    while chromosome < len(individual):
        previousCoor = [coordList[chromosome][0],coordList[chromosome][1]]
        #Modify position based on the current chromosome.
        if individual[chromosome] == 0:
            previousCoor[1] -= 1
            coordList[chromosome + 1][0]= previousCoor[0]
            coordList[chromosome + 1][1]= previousCoor[1]
        if individual[chromosome] == 1:
            previousCoor[0] += 1
            coordList[chromosome + 1][0]= previousCoor[0]
            coordList[chromosome + 1][1]= previousCoor[1]
        if individual[chromosome] == 2:
            previousCoor[1] += 1
            coordList[chromosome + 1][0]= previousCoor[0]
            coordList[chromosome + 1][1]= previousCoor[1]
        if individual[chromosome] == 3:
            previousCoor[0] -= 1
            coordList[chromosome + 1][0]= previousCoor[0]
            coordList[chromosome + 1][1]= previousCoor[1]
        chromosome += 1
    return coordList

def crossover(orderedPopulationList):
#Performs breeding on a list of individuals [[[individual], fitness],...].
#Combining random individuals from the top proportion of fitnesses chromosomes at a random point.
    newPopulation = []
    count = 0
    while count < len(orderedPopulationList):
        newMember = []
        point = random.randrange(0, len(topFracPop(orderedPopulationList, 2.0)[0])) #random crossover point.
        left = random.randrange(0,len(topFracPop(orderedPopulationList, 2.0))) #Choose an individual in the top half of the population
        right = random.randrange(0,len(topFracPop(orderedPopulationList, 2.0)))
        firstHalf = (topFracPop(orderedPopulationList, 2.0)[left][: point]) #Obtain the individuals genes up until the crossover point
        secondHalf = (topFracPop(orderedPopulationList, 2.0)[right][point :])
        newMember.append(firstHalf + secondHalf) #Append newborn to new population
        newPopulation.append(newMember[0])
        count += 1
    return newPopulation

def mutate(crossed, percentage_mutated):
#Mutates some individuals in a population by randomly changing a chromosome.
    num = round((percentage_mutated/100.0) * len(crossed))
    count = 0
    while count < num:
        crossed[random.randrange(0, len(crossed))][random.randrange(0, len(crossed[0]))] = random.randrange(0, 4)
        count += 1
    return crossed

# Non-Bayesian Version of the Genetic Algorithm

Here is the traditional version of the Genetic Algorithm. It was used to find effective parsmeter settings, and as a benchmark to see how effective the upcoming Bayesian version is.

![](gaimg.png)

In [5]:
def basicMazeGA(maze, popSize, mutateRate, individualLen, verbose):
    iteration = 1
    currBest = [[2,2,2,2,2,2],2] #dummy data to get it started.
    while currBest[1] > 0: #while the fitness is not 0.
        individualCount = 0
        disorderedList = []
        while individualCount < popSize: #for each member of the population.
            indivAndFitness = [[None],None]
            if iteration == 1:
                individual = generateIndividual(individualLen)
            else:
                individual = mutated[individualCount]
            coors = allCoordsVisited(maze, individual)
            fitness = distance(maze, coors)
            indivAndFitness[0] = individual
            indivAndFitness[1] = fitness
            disorderedList.append(indivAndFitness)
            individualCount = individualCount + 1
        fitSortedList = sorted(disorderedList, key=itemgetter(1))
        currBest = fitSortedList[0]
        if verbose == 1:
            print("iteration: ", iteration)
        if currBest[1] == 0:
            print("Generation:")
            print(iteration)
            print("Individual:")
            print(currBest[0])
            #break
        crossed = crossover(fitSortedList)
        mutated = mutate(crossed, mutateRate)
        if currBest[1] == 0:
            return iteration
        iteration += 1

#Running the algorithm once in verbose mode.
basicMazeGA(maze, popSize, mutateRate, individualLen, 1)

iteration:  1
iteration:  2
iteration:  3
iteration:  4
iteration:  5
iteration:  6
iteration:  7
iteration:  8
iteration:  9
iteration:  10
iteration:  11
iteration:  12
iteration:  13
iteration:  14
iteration:  15
iteration:  16
iteration:  17
iteration:  18
iteration:  19
Generation:
19
Individual:
[2, 1, 2, 1, 1, 2, 2, 2, 2, 1, 1, 0, 0, 3, 1, 2, 3, 0, 0, 1]


19

**Now we run the algorithm a number of times to obtain the mean and variance so we can compare this to the Bayesian version**

# Bayesian-specific functions

Here are the additional functions needed to run the Bayesian version of the Genetic Algorithm.

**computePosterior** calculates the Posterior distribution of programs in a generation from their prior
distribution and likelihood for the fitness data observed using Bayes theorem, where the posterior probability of A given B  $P(A | B)$  is the result of multiplying the likelihood $P(B | A)$ by the prior $P(A)$, and dividing this product by the Marginalization ${P(B)}$ :

$$P(A | B)=\frac{P(B | A)  P(A)}{P(B)}$$

For our purposes, the formula will look as follows:

$$ P_{g}\left(A_{i}^{g} | D_{i}^{g}\right)=\frac{P\left(D_{i}^{g} | A_{i}^{g}\right) P_{g-1}\left(A_{i}^{g} | D_{i}^{g-1}\right)}{\sum_{j=1}^{M} P\left(D_{j}^{g} | A_{j}^{g}\right) P_{g-1}\left(A_{j}^{g} | D_{j}^{g-1}\right)} $$

Where g is the current generation number, M is the population size, and i is the individual in the population. The formula calculates the probability of obtaining a certain individual A in a generation given the fitness data D for that generation. Note that our prior is the previous generation's posterior.

**generateFlatDistro** is used as our initial prior belief for the first generation (before we have any knowledge what our population looks like). It simply returns a list of the same size as the population containing $1/popSize$ for each entry. 

**bayesSelectProb** This function returns a list with the probability that an individual is selected for passing on it's genetic information. It calculates the ratio of the current posterior to the old posterior for each individual, and then chooses the minimum between this ratio and 1 (so probabilities can't be higher than 100%). The **decision** function is used to evaluate these probabilities in the main body of the program.

**bayesCrossover** This is similar to the original crossover function above, but performs crossovers over the whole population, not just a top fraction of them (no sorting required).

**computeLikelihood** calculates the likelihood function for the current population. As suggested by Zhang's paper, the likelihood function implemented is the Boltzmann distribution:

$$P(D | A)=\frac{1}{Z_{D}(\beta)} \exp \left(-\beta F_{D}\right)$$

Where where $F_{D}$ is an error function (in this case our Manhattan distance fitness, which worked better than pythons math.erf), $\beta$ controls the variance of the noise (I found this worked well just being set to 1), and $Z_{D}$ is a normalization factor ensuring the elements in the array form probability distribution by summing to 1.
The likelihood factor gives preference to programs that have less error for the fitness cases.


In [7]:
def generateFlatDistro(popSize):
#generates a uniform distribution (used as our first prior)
    return [(1.0/popSize) for i in range(popSize)]

def bayesCrossover(populationList):
#Performs breeding on a list of individuals, by combining random individuals
    newPopulation = []
    count = 0
    while count < len(populationList):
        newMember = []
        point = random.randrange(0, len(populationList[0][0])) #random crossover point.
        left = random.randrange(0, len(populationList))
        right = random.randrange(0, len(populationList))
        firstHalf = populationList[left][0][: point]
        secondHalf = populationList[right][0][point :] #list[start:stop] [#:]
        newMember.append(firstHalf + secondHalf) #Append newborn to new population
        newPopulation.append(newMember[0])
        count += 1
    return newPopulation

def computeLikelihood(disorderedList, noiseVar): #probably should redo this with numpy if using a very large populations.
#Computes the boltzmann distribution to model our likelihood function.
    exps = []
    likelihoods = []
    sumExp = 0
    for i in disorderedList:
        exps.append(math.exp(-1.0 * i[1] * noiseVar)) #Calculate Boltzmann distro
    sumExp = sum(exps)
    for i in exps:
        likelihoods.append(i / sumExp) #Convert into a probability distribution
    return likelihoods

def computePosterior(priors, likelihoods):
#Calculate posterior distributions given priors and likelihoods.
    posteriors = []
    count = 0
    while count < len(priors):
        thisLikelihood = likelihoods[count]
        thisPrior = priors[count]
        numerator = thisLikelihood * thisPrior
        count2 = 0
        denominator = 0
        while count2 < len(priors):
            denominator += likelihoods[count2] * priors[count2]
            count2 += 1
        posterior = numerator / denominator
        posteriors.append(posterior)
        count += 1
    return posteriors

def decision(probability):
#returns true with a probability
    return random.random() < probability

def bayesSelectProb(posteriorsOld, posteriorsNew):
#Given the previous generations, and current posteriors, returns the probability of acceptance for each individual.
    probSelected = []
    count = 0
    while count < len(posteriorsOld):
        ratio = posteriorsNew[count]/posteriorsOld[count]
        probSelected.append(min(1.0, ratio))
        count = count + 1
    return probSelected

# Bayesian Genetic Algorithm

The algorithm is similar in style to the diagram presented for the traditional genetic algorithm, with some additions.

1. Initialize the first generation of genetic programs, calculate their fitness, and compute the posterior.

2. Perform breeding and mutation of the new generation, and compute the new fitness values.

3. Update the prior to be the previous generations posterior.

4. Compute the new posterior with the new fitness values.

5. Select which of the newly bred individuals are kept by comparing the current and the previous posteriors. Individuals that are not kept are replaced with max-pooling. Store the old posterior.

6. Check if we have solved the maze. If so, we're done. If not, go to step 2.

In [11]:
def bayesianMazeGA(maze, popSize, mutateRate, individualLen, verbose):
    iteration = 1
    individualCount = 0
    disorderedList = []

    #We initialize the first generation outside the main loop
    while individualCount < popSize: #for each member of the population.
        indivAndFitness = [[None],None]
        individual = generateIndividual(individualLen)
        coors = allCoordsVisited(maze, individual)
        fitness = distance(maze, coors)
        indivAndFitness[0] = individual
        indivAndFitness[1] = fitness
        disorderedList.append(indivAndFitness)
        individualCount = individualCount + 1

    currBest = sorted(disorderedList, key=itemgetter(1))[0]
    priors = generateFlatDistro(popSize) #initialize to flat prior
    likelihoods = computeLikelihood(disorderedList, 1.0)
    posteriorsOld = computePosterior(priors, likelihoods)

    #Loop process for the rest of the generations until a solution is found
    while currBest[1] > 0:

        crossed = bayesCrossover(disorderedList)
        #orderedList = sorted(disorderedList, key=itemgetter(1))
        #crossed = crossover(orderedList) 
        mutated = mutate(crossed, mutateRate)
        disorderedList = []

        individualCount = 0
        while individualCount < popSize:
            indivAndFitness = [[None],None]
            individual = mutated[individualCount]
            coors = allCoordsVisited(maze, individual)
            fitness = distance(maze, coors)
            indivAndFitness[0] = individual
            indivAndFitness[1] = fitness
            disorderedList.append(indivAndFitness)
            individualCount = individualCount + 1

        priors = posteriorsOld #update prior
        likelihoods = computeLikelihood(disorderedList, 1.0)
        posteriorsNew = computePosterior(priors, likelihoods)
        probs = bayesSelectProb(posteriorsOld, posteriorsNew) #get possibility of acceptance for new generation
 
        #selection process
        i = 0
        selected = []
        while i < popSize:
            currProb = probs[i]
            if decision(currProb) == True: #Decide if an individual is accepted or rejected for the next generation
                selected.append(mutated[i])
            else: #If rejected, replace with a max-pooled individual.
                selected.append(sorted(disorderedList, key=itemgetter(1))[random.randrange(0,0.03*popSize)][0]) #If newbred is rejected, then just use parent in that position of array is kept
            i = i + 1

        posteriorsOld = posteriorsNew
            
        currBest = sorted(disorderedList, key=itemgetter(1))[0] #Get best population member by fitness.
        #print("Current best: ", currBest)
        if verbose == 1:
            print("iteration: ", iteration)
        if currBest[1] == 0:
            print("Generation:")
            print(iteration)
            print("Individual:")
            print(currBest[0])    

        disorderedList = [] #reset as needs to be [selected[i], fitness]
        for i in selected:
            disorderedList.append([i, 'dummyFitness'])
        if currBest[1] == 0:
            return iteration        
        iteration += 1
        
#Running the algorithm once in verbose mode.
bayesianMazeGA(maze, popSize, mutateRate, individualLen, 1)

iteration:  1
iteration:  2
iteration:  3
iteration:  4
iteration:  5
iteration:  6
iteration:  7
iteration:  8
Generation:
8
Individual:
[2, 1, 2, 1, 1, 2, 2, 3, 1, 2, 2, 1, 1, 0, 0, 2, 3, 1, 2, 1]


8

# Comparison of the two versions

In [17]:
numReps = 20 #How many trials to run
iters = []
trial = 0
times = []
while trial < numReps:
    trial += 1
    print("Trial number: ", trial)
    start = time.time()
    x = basicMazeGA(maze, popSize, mutateRate, individualLen, 0)
    end = time.time()
    print("-----------------------------------------------------")
    iters.append(x)
    times.append(end - start)
print("Mean number of generations taken to solve the maze: ", statistics.mean(iters))
print("Median number of generations taken to solve the maze: ", statistics.median(iters))
print("Mean time in seconds taken to solve the maze: ", statistics.mean(times))
print("Median time in seconds taken to solve the maze: ", statistics.median(times))

Trial number:  1
Generation:
16
Individual:
[2, 1, 2, 1, 3, 1, 1, 2, 2, 2, 2, 1, 1, 0, 0, 3, 2, 0, 2, 2]
-----------------------------------------------------
Trial number:  2
Generation:
27
Individual:
[1, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1, 1, 3, 0, 1, 3, 3, 1, 2, 2]
-----------------------------------------------------
Trial number:  3
Generation:
34
Individual:
[2, 1, 2, 1, 1, 2, 0, 2, 2, 2, 2, 1, 1, 0, 3, 3, 1, 0, 0, 1]
-----------------------------------------------------
Trial number:  4
Generation:
18
Individual:
[1, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1, 0, 1, 3, 3, 3, 0, 3, 1, 1]
-----------------------------------------------------
Trial number:  5
Generation:
13
Individual:
[2, 1, 2, 1, 1, 2, 2, 3, 1, 0, 2, 2, 2, 1, 1, 0, 2, 0, 0, 0]
-----------------------------------------------------
Trial number:  6
Generation:
21
Individual:
[1, 2, 2, 0, 2, 1, 1, 2, 2, 2, 2, 1, 3, 1, 1, 0, 3, 2, 2, 2]
-----------------------------------------------------
Trial number:  7
Generation:
140
Individual:
[

In [18]:
numReps = 20 #How many trials to run
iters = []
trial = 0
times = []
while trial < numReps:
    trial += 1
    print("Trial number: ", trial)
    start = time.time()
    x = bayesianMazeGA(maze, popSize, mutateRate, individualLen, 0)
    end = time.time()
    print("-----------------------------------------------------")
    iters.append(x)
    times.append(end - start)
print("Mean number of generations taken to solve the maze: ", statistics.mean(iters))
print("Median number of generations taken to solve the maze: ", statistics.median(iters))
print("Mean time in seconds taken to solve the maze: ", statistics.mean(times))
print("Median time in seconds taken to solve the maze: ", statistics.median(times))

Trial number:  1
Generation:
8
Individual:
[2, 1, 2, 1, 1, 2, 2, 3, 1, 2, 2, 1, 1, 3, 1, 0, 0, 2, 2, 1]
-----------------------------------------------------
Trial number:  2
Generation:
9
Individual:
[1, 2, 2, 1, 1, 2, 2, 2, 2, 1, 3, 1, 3, 1, 1, 0, 3, 0, 2, 2]
-----------------------------------------------------
Trial number:  3
Generation:
5
Individual:
[2, 1, 2, 1, 1, 2, 2, 2, 2, 1, 1, 3, 1, 0, 0, 1, 2, 3, 1, 1]
-----------------------------------------------------
Trial number:  4
Generation:
28
Individual:
[2, 1, 2, 1, 1, 2, 2, 2, 2, 0, 0, 2, 2, 1, 1, 0, 2, 1, 2, 3]
-----------------------------------------------------
Trial number:  5
Generation:
59
Individual:
[2, 1, 2, 1, 1, 2, 0, 2, 2, 3, 1, 2, 2, 1, 1, 0, 0, 2, 1, 3]
-----------------------------------------------------
Trial number:  6
Generation:
7
Individual:
[2, 1, 2, 1, 1, 3, 1, 2, 2, 2, 2, 1, 1, 0, 2, 0, 2, 1, 3, 3]
-----------------------------------------------------
Trial number:  7
Generation:
11
Individual:
[1, 2,

# Conclusion

As can be seen from the output of the code above, using bayesian statistics to select the population members which pass on their chromosomes in a genetic algorithm is beneficial to the convergence time of the algorithm. The Bayesian version of the genetic algorithm outperforms the traditional version by using the information gained in the previous generations to generate better offspring, and to determine when individuals needs replacement through max-pooling. Both the number of generations taken to reach the solution and the processor time taken to run the algorithm are reduced by using these modifications.